### Dependencies

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC

import gradio as gr

### Data 

In [4]:
path = os.getcwd().replace('main','Resources') + '\\SMSSpamCollection.csv'
sms_txt_df = pd.read_csv(path)

sms_txt_df.head()

,label,text_message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


### sms Classification Function

In [6]:
def sms_classification(sms_txt_df):
    """
    Perform SMS classification using a pipeline with TF-IDF vectorization and Linear Support Vector Classification.

    Parameters:
    - sms_text_df (pd.DataFrame): DataFrame containing 'text_message' and 'label' columns for SMS classification.

    Returns:
    - text_clf (Pipeline): Fitted pipeline model for SMS classification.

    This function takes a DataFrame with 'text_message' and 'label' columns, splits the data into
    training and testing sets, builds a pipeline with TF-IDF vectorization and Linear Support Vector
    Classification, and fits the model to the training data. 
    The fitted pipeline is returned to make future predictions.
    """
    # Set the features variable to the text message column.
    X= sms_txt_df['text_message']
    # Set the target variable to the "label" column.
    y = sms_txt_df['label']

    # Split data into training and testing and set the test_size = 33%
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.33, random_state=42)

    # Build a pipeline to transform the test set to compare to the training set.
    text_clf = Pipeline([('tfidf', TfidfVectorizer(stop_words='english')),('clf', LinearSVC())])
    

    # Fit the model to the transformed training data and return model.
    return text_clf.fit(X_train,y_train)

### Generating text classfication model

In [8]:
text_clf = sms_classification(sms_txt_df)

### sms Prediction Function

In [10]:
# Create a function called `sms_prediction` that takes in the SMS text and predicts the whether the text is "not spam" or "spam". 
# The function should return the SMS message, and say whether the text is "not spam" or "spam".
def sms_prediction(text):
    """
    Predict the spam/ham classification of a given text message using a pre-trained model.

    Parameters:
    - text (str): The text message to be classified.

    Returns:
    - str: A message indicating whether the text message is classified as spam or not.

    This function takes a text message and a pre-trained pipeline model, then predicts the
    spam/ham classification of the text. The result is a message stating whether the text is
    classified as spam or not.
    """
    # Create a variable that will hold the prediction of a new text.
    y_preds = text_clf.predict([text])
    # Using a conditional if the prediction is "ham" return the message:
    if y_preds[0] == 'ham':
        return f'The text message: "{text}", is not spam.' 
    else:
        return f'The text message: "{text}", is spam.'
    
    

### sms classification App

In [13]:
theme = gr.themes.Glass()
with gr.Blocks(theme = theme) as sms_app:
    text = gr.Textbox(label='What is the next message you want to test?')
    output = gr.Textbox(label='Our app has determined:',
                       lines = 10)
    gr.Interface(fn=sms_prediction, 
                       inputs=text, 
                       outputs=output
                      )

### App Launch

In [15]:
sms_app.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


## Test the following text messages. 

---

1. You are a lucky winner of $5000!
2. You won 2 free tickets to the Super Bowl.
3. You won 2 free tickets to the Super Bowl text us to claim your prize.
4. Thanks for registering. Text 4343 to receive free updates on medicare.

---

### Test Results

#### 1) **The text message: "You are a lucky winner of $5000!", <font color='blue'>is not spam</font>.**

#### 2) **The text message: "You won 2 free tickets to the Super Bowl.",  <font color='blue'>is not spam</font>.**

#### 3) **The text message: "You won 2 free tickets to the Super Bowl text us to claim your prize.", <font color='red'>is spam</font>.**

#### 4) **The text message: "Thanks for registering. Text 4343 to receive free updates on medicare.", <font color='red'>is spam</font>.**